# Re: Issue-1 connectedness analyses #1
---

**Author:** Chen Weng

**Date:** Febuary 25, 2024

## Description

This Jupyter Notebook clarify the connectedness analysis in Fig. 1f

---

In [2]:
library(Matrix)
library(dplyr)
library(redeemR)
library(igraph)

In [177]:
Dir="/lab/solexa_weissman/cweng/Projects/MitoTracing_Velocity/SecondaryAnalysis/reproducibility_pub" # Change this to your actual path **/reproducibility_pub

In [4]:
## Load mtscATAC(no_enrich) mgatk output for Young1 HPC
Young1_HPC.mgatk.stats<-read.delim(paste(Dir,"/data/redeemV_final/Young1.T1.HPC.NoEnrich.mgatk.final/final/DN4_HSPC.variant_stats.tsv.gz",sep=""))
Young1_HPC.cell_heteroplasmic<-read.delim(paste(Dir,"/data/redeemV_final/Young1.T1.HPC.NoEnrich.mgatk.final/final/DN4_HSPC.cell_heteroplasmic_df.tsv.gz",sep=""))

In [86]:
## Load redeem output for Young1 HPC (No-enrichment data)
Young1_HPC_mitoTracing.sensitive<-readRDS(paste(Dir,"/data/redeemR_object_oldversion/DN4_HPC_mitoTracing.sensitive",sep=""))

In [88]:
# mgatk data filtering, binarization,and removing the two highly abundant variants 310 and 3109
Young1_mgatk_Selected_V<-subset(Young1_HPC.mgatk.stats,strand_correlation>0.6 & vmr>0.01 & n_cells_conf_detected>=3)
Young1_mgatk_Selected_V<-subset(Young1_mgatk_Selected_V,!position %in% c(310,3109))
SelectV.format<-paste("X",gsub(">",".",Young1_mgatk_Selected_V$variant),sep="")
mgatk.selected.mtx<-Young1_HPC.cell_heteroplasmic[,c("X",SelectV.format)] %>% tibble::column_to_rownames("X")
mgatk.selected.mtx[is.na(mgatk.selected.mtx)]<-0
mgatk.selected.mtx[mgatk.selected.mtx!=0]<-1
mgatk.selected.MTX<-as(as.matrix(mgatk.selected.mtx), "sparseMatrix")

In [89]:
## Extract the redeem binary variant matrix, removing the two highly abundant variants 310T>C and 3109T>C
Young1_HPC.Cts.Mtx.bi<-Young1_HPC_mitoTracing.sensitive@Cts.Mtx.bi[,-match(c("Variants310TC","Variants3109TC"),colnames(Young1_HPC_mitoTracing.sensitive@Cts.Mtx.bi))]


In [176]:
## function to count the connectedness (adjacency matrix), or the number of cells sharing more than n variants with the given cell
CountOverlap_Adj<-function(M,n=2){
    require(Matrix)
    # Total <- Matrix::rowSums(M)
    a <- M %*% Matrix::t(M)
    a[a<=n]<-0
    a[a>n]<-1
    return(a)
}

In [91]:
## Adjacency matrix,more than 2 variants shared
mgatk_n2.adj.mtx<-CountOverlap_Adj(mgatk.selected.MTX,n=2) 
redeem_n2.adj.mtx<-CountOverlap_Adj(Young1_HPC.Cts.Mtx.bi,n=2)

## Adjacency matrix,more than 1 variants shared
## This is reported in Fig. 1f
mgatk_n1.adj.mtx<-CountOverlap_Adj(mgatk.selected.MTX,n=1) 
redeem_n1.adj.mtx<-CountOverlap_Adj(Young1_HPC.Cts.Mtx.bi,n=1)

## Adjacency matrix,more than 0 variants shared
mgatk_n0.adj.mtx<-CountOverlap_Adj(mgatk.selected.MTX,n=0) 
redeem_n0.adj.mtx<-CountOverlap_Adj(Young1_HPC.Cts.Mtx.bi,n=0)

In [92]:
## Count the number of cells that share more than 2 variants (The connectdedness is self-inclusive)
rowSums(mgatk_n2.adj.mtx) %>% summary
rowSums(redeem_n2.adj.mtx) %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.0000  0.0000  0.2996  0.0000 13.0000 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   0.00    1.00    1.00   12.01    6.00  256.00 

In [93]:
## Count the number of cells that share more than 1 variants (The connectdedness is self-inclusive)
## This is reported in Fig. 1f
rowSums(mgatk_n1.adj.mtx) %>% summary
rowSums(redeem_n1.adj.mtx) %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   0.000   0.000   4.017   2.000 179.000 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    0.0     3.0    16.0    50.8    74.0   508.0 

In [94]:
## Count the number of cells that share more than 0 variants (The connectdedness is self-inclusive)
rowSums(mgatk_n0.adj.mtx) %>% summary
rowSums(redeem_n0.adj.mtx) %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    0.0     0.0    42.0   182.2   165.0  1340.0 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    0.0   156.0   336.0   391.8   565.0  1823.0 

> We tested several different "n" to check the robustness of the comparison on connectedness.
The difference between before (via mgatk) and after enrichment (via redeem) is consistent.
We reported the redeem_n1.adj.mtx

In Fig. 1f, The adjacency matrix is then converted into igraph and output into software grephi for visulization

In [163]:
#### Re Caleb's note  https://github.com/chenweng1991/redeem_reproducibility/issues/1
#------
# Need help from here:
# how does one compute the cell-cell connectedness?
#------
Cts.Mtx.bin<-Young1_HPC.Cts.Mtx.bi
# Caleb attempt:
# loop over each cell 
sapply(1:dim(Cts.Mtx.bin)[1], function(i){
  
  # determine what mutations that individual cell has
  variant_indicies <- as.numeric(which(Cts.Mtx.bin[i,] > 0))
  
  # subset full matrix to only those mutations (and remove index cell)
  Cts.Mtx.bin.ss <- Cts.Mtx.bin[-i,variant_indicies, drop = FALSE]
  
  # count the number of cells that have one or more of the mutations
  # found in the original cell
  # This should be the connectedness, right? 
  sum(rowSums(Cts.Mtx.bin.ss) >= 1)
  
}) %>% summary() 

### This measure is equivalent to redeem_n0.adj.mtx
### In Fig. 1f, We actually reported the connectedness with a more stringent threadhold redeem_n1.adj.mtx. 


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    0.0   155.0   335.0   390.8   564.0  1822.0 

In that regard, to clarify the definition of connectedness in Fig. 1f: </br>
*The connectedness is defined as the number of “neighbor” cells that share more than 1 mtDNA mutation with any given cell.*